In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import time
from src.logger import logging
from src.utils.helper import get_osrm_distance
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

In [2]:
def create_osrm_distance_mat(df):
    try:
        # Validate required columns
        required_columns = {"ADDRESS", "LATITUDE", "LONGITUDE"}
        if not required_columns.issubset(df.columns):
            logging.error("Missing required columns in CSV file.")
            raise ValueError("CSV file must contain 'ADDRESS', 'LATITUDE', and 'LONGITUDE' columns")
        
        # Remove duplicates and extract necessary data
        unique_locations_df = df.drop_duplicates(subset=["ADDRESS"])[["ADDRESS", "LATITUDE", "LONGITUDE"]]
        location_names = unique_locations_df["ADDRESS"].to_list()
        locations_gps = list(zip(unique_locations_df["LATITUDE"], unique_locations_df["LONGITUDE"]))
        
        num_locations = len(location_names)
        distance_matrix = np.zeros((num_locations, num_locations))
        
        logging.info(f"Processing {num_locations} locations for distance matrix computation.")
        
        # Compute distance matrix
        for i in range(num_locations):
            for j in range(i + 1, num_locations):  # Avoid duplicate calculations
                try:
                    dis = get_osrm_distance(locations_gps[i], locations_gps[j])
                    dis = round(dis,2)
                    distance_matrix[i, j] = dis
                    distance_matrix[j, i] = dis
                    time.sleep(0.2)  # Prevent rate limiting
                except Exception as e:
                    logging.error(f"Error computing distance between {location_names[i]} and {location_names[j]}: {str(e)}")
                    distance_matrix[i, j] = distance_matrix[j, i] = float('inf')  # Set as unreachable if an error occurs
        
        logging.info("Distance matrix computation completed successfully.")
        return distance_matrix, location_names
    
    except Exception as e:
        logging.critical(f"Unexpected error in distance matrix creation: {str(e)}")
        raise

In [3]:
def create_data_model(
    distance_matrix=None,
    locations=None,
    depot=0,
    shop_demands=None,
    vehicle_capacities=None,
    vehicle_restricted_locations=None,
    vehicle_restricted_roads=None,
    same_route_locations=None,
):
    return {
        "distance_matrix": distance_matrix or [],
        "locations": locations or [],
        "depot": depot,
        "shop_demands": shop_demands or {},
        "vehicle_capacities": vehicle_capacities or {},
        "vehicle_restricted_locations": vehicle_restricted_locations or {},
        "vehicle_restricted_roads": vehicle_restricted_roads or {},
        "same_route_locations": same_route_locations or [],
    }

In [5]:
def vrp_manager(data):
    try:
        logging.info("Initializing TSP Manager with depot location.")
        distance_matrix = data.get('distance_matrix')
        depot = data.get('depot')
        num_vehicles = len(data.get('vehicle_capacities'))
        manager = pywrapcp.RoutingIndexManager(
            len(distance_matrix), num_vehicles, depot
        ) 
        
        logging.info("TSP Manager successfully created.")
        return manager
    except ValueError as ve:
        logging.error(f"Depot location not found in locations list: {str(ve)}")
    except Exception as e:
        logging.critical(f"Unexpected error in TSP Manager creation: {str(e)}")

In [ ]:
def create_vehicle_cost_callback(vehicle_id, data, manager):
    """
    Returns a transit callback that accounts for:
    - Vehicle-specific restricted roads
    - General restricted roads
    - Default distance from the distance matrix
    """

    def transit_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)

        # Vehicle-specific restricted roads
        if (from_node, to_node) in data.get("vehicle_restricted_roads", {}).get(vehicle_id, []):
            logging.warning(f"Vehicle {vehicle_id} restricted from traveling {from_node} -> {to_node}. Applying high cost.")
            return int(1e6)  # High penalty cost

        # General restricted roads
        if (from_node, to_node) in data.get("restricted_roads", []):
            logging.warning(f"Route {from_node} -> {to_node} is generally restricted. Applying high cost.")
            return int(1e6)  # High penalty cost

        # Default travel cost from distance matrix
        return int(data["distance_matrix"][from_node][to_node])

    return transit_callback


def create_demand_callback(data, manager):
    """Returns a callback function to fetch shop demands correctly."""

    def demand_callback(from_index):
        """Fetches demand value for a given location index."""
        from_node = manager.IndexToNode(from_index)
        return data["shop_demands"].get(data["locations"][from_node], 0)  

    return demand_callback



def add_capacity_constraints(routing, manager, data):
    """Ensures vehicle capacity constraints are applied correctly."""
    
    demand_callback = create_demand_callback(data, manager)
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # No slack
        list(data["vehicle_capacities"].values()),  
        True,  # Start cumul to zero
        "Capacity",
    )

    logging.info("Capacity constraints added successfully.")


    logging.info("Capacity constraints added successfully.")


def add_distance_constraints(routing, transit_callback_index):
    """
    Adds distance constraints to prevent exceeding a maximum travel distance.
    """
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # No slack
        3000,  # Default vehicle max travel distance
        True,  # Start cumul to zero
        dimension_name,
    )

    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    
    logging.info("Distance constraints added successfully.")


def add_same_route_constraints(routing, manager, data):
    """
    Ensures that paired locations (pickup & delivery) are assigned to the same vehicle.
    If no such constraints exist, the function safely skips execution.
    """
    same_route_locations = data.get("same_route_locations", [])
    if not same_route_locations:
        logging.info("No same-route constraints provided. Skipping this step.")
        return

    for request in same_route_locations:
        try:
            pickup_index = manager.NodeToIndex(request[0])
            delivery_index = manager.NodeToIndex(request[1])
            routing.solver().Add(
                routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index)
            )
            logging.info(f"Same-route constraint added: {request[0]} <-> {request[1]}")
        except Exception as e:
            logging.error(f"Error adding same-route constraint for {request}: {str(e)}")


def add_vehicle_restrictions(routing, manager, data):
    """
    Prevents specific vehicles from visiting certain locations.
    If no such restrictions exist, the function safely skips execution.
    """
    vehicle_restricted_locations = data.get("vehicle_restricted_locations", {})
    if not vehicle_restricted_locations:
        logging.info("No vehicle-specific restricted locations provided. Skipping this step.")
        return

    for vehicle_id, locations in vehicle_restricted_locations.items():
        for location in locations:
            try:
                index = manager.NodeToIndex(location)
                routing.solver().Add(routing.VehicleVar(index) != vehicle_id)
                logging.info(f"Vehicle {vehicle_id} restricted from visiting location {location}.")
            except Exception as e:
                logging.error(f"Error restricting vehicle {vehicle_id} from location {location}: {str(e)}")


def solve_vrp(data):
    """
    Solves the Vehicle Routing Problem (VRP) given distance matrix, demands, and constraints.
    """

    # Initialize OR-Tools Manager & Routing Model
    manager = vrp_manager(data=data)
    routing = pywrapcp.RoutingModel(manager)

    num_vehicles = len(data.get("vehicle_capacities", []))
    if num_vehicles == 0:
        logging.error("No vehicles available for VRP.")
        raise ValueError("No vehicles available for VRP.")

    # Register vehicle-specific cost callbacks
    for vehicle_id in range(num_vehicles):
        transit_callback = create_vehicle_cost_callback(vehicle_id, data, manager)
        transit_callback_index = routing.RegisterTransitCallback(transit_callback)
        routing.SetArcCostEvaluatorOfVehicle(transit_callback_index, vehicle_id)

    # Apply constraints
    add_capacity_constraints(routing, manager, data)
    add_distance_constraints(routing, transit_callback_index)
    add_same_route_constraints(routing, manager, data)
    add_vehicle_restrictions(routing, manager, data)

    # Configure search parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )

    logging.info("Solving VRP...")
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        logging.info("VRP Solution Found!")
    else:
        logging.warning("No solution found for VRP.")

    return manager, routing, solution


In [ ]:
def get_vrp_solution(data):
    """
    Extracts solution details for VRP and returns structured route paths.
    
    Returns:
        dict: Containing each vehicle's route details with distance & load.
    """
    
    manager, routing, solution = solve_vrp(data=data)

    if not solution:
        logging.error("No solution found!")
        return {"message": "No solution found!"}

    total_distance = 0
    total_load = 0
    routes = {}
    
    num_vehicles = len(data.get('vehicle_capacities'))

    for vehicle_id in range(num_vehicles):
        index = routing.Start(vehicle_id)
        route_path = []
        route_distance = 0
        route_load = 0
        vehicle_details = {
            "vehicle_id": vehicle_id,
            "route": [],
            "total_distance": 0,
            "total_load": 0
        }

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            location_name = data['locations'][node_index]  
            demand = data['shop_demands'].get(location_name, 0)  
            route_load += demand  

            route_path.append({
                "location": location_name,  
                "current_load": route_load
            })
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        # Add last depot stop
        route_path.append({"location": data['locations'][manager.IndexToNode(index)], "current_load": route_load})
        
        # Store route details
        vehicle_details["route"] = route_path
        vehicle_details["total_distance"] = route_distance
        vehicle_details["total_load"] = route_load
        routes[f"vehicle_{vehicle_id}"] = vehicle_details

        # Update overall totals
        total_distance += route_distance
        total_load += route_load

        logging.info(f"Route for vehicle {vehicle_id}: {route_path}")
        logging.info(f"Distance: {route_distance} km, Load: {route_load}")

    logging.info(f"Total distance of all routes: {total_distance} km")
    logging.info(f"Total load of all routes: {total_load}")

    return {
        "routes": routes,
        "total_distance": total_distance,
        "total_load": total_load
    }


In [24]:
df = pd.read_csv('../data/sheets/test.csv')

In [25]:
df.columns

Index(['INVOICE NO', 'CUSTOMER NAME', 'ADDRESS', 'Invoice Value', 'Rep',
       'LATITUDE', 'LONGITUDE', 'DEMAND'],
      dtype='object')

In [26]:
distance_matrix, location_names = create_osrm_distance_mat(df)

In [27]:
distance_matrix, location_names

(array([[  0.  ,  53.53, 117.15,  76.13, 102.99, 132.7 ],
        [ 53.53,   0.  , 109.71,  79.49, 157.07, 169.49],
        [117.15, 109.71,   0.  ,  57.75, 187.27, 169.94],
        [ 76.13,  79.49,  57.75,   0.  , 129.61, 112.28],
        [102.99, 157.07, 187.27, 129.61,   0.  ,  29.79],
        [132.7 , 169.49, 169.94, 112.28,  29.79,   0.  ]]),
 ['Nochchiyagama',
  'Medawachchiya',
  'Medirigiriya',
  'Dambulla',
  'Chilaw',
  'Walipennagahamula'])

In [28]:
data = create_data_model(distance_matrix=distance_matrix.tolist(), locations=location_names)

In [29]:
data

{'distance_matrix': [[0.0, 53.53, 117.15, 76.13, 102.99, 132.7],
  [53.53, 0.0, 109.71, 79.49, 157.07, 169.49],
  [117.15, 109.71, 0.0, 57.75, 187.27, 169.94],
  [76.13, 79.49, 57.75, 0.0, 129.61, 112.28],
  [102.99, 157.07, 187.27, 129.61, 0.0, 29.79],
  [132.7, 169.49, 169.94, 112.28, 29.79, 0.0]],
 'locations': ['Nochchiyagama',
  'Medawachchiya',
  'Medirigiriya',
  'Dambulla',
  'Chilaw',
  'Walipennagahamula'],
 'depot': 0,
 'shop_demands': {},
 'vehicle_capacities': {},
 'vehicle_restricted_locations': {},
 'vehicle_restricted_roads': {},
 'same_route_locations': []}

In [51]:
if "DEMAND" in df.columns:
    demand_list = df['DEMAND'].to_list()
    data['shop_demands'] = {}
    for i, loc in enumerate(df['ADDRESS']):
        print(loc)
        data['shop_demands'][loc] = demand_list[i]

Nochchiyagama
Medawachchiya
Medirigiriya
Dambulla
Chilaw
Walipennagahamula
Walipennagahamula


In [52]:
data

{'distance_matrix': [[0.0, 53.53, 117.15, 76.13, 102.99, 132.7],
  [53.53, 0.0, 109.71, 79.49, 157.07, 169.49],
  [117.15, 109.71, 0.0, 57.75, 187.27, 169.94],
  [76.13, 79.49, 57.75, 0.0, 129.61, 112.28],
  [102.99, 157.07, 187.27, 129.61, 0.0, 29.79],
  [132.7, 169.49, 169.94, 112.28, 29.79, 0.0]],
 'locations': ['Nochchiyagama',
  'Medawachchiya',
  'Medirigiriya',
  'Dambulla',
  'Chilaw',
  'Walipennagahamula'],
 'depot': 0,
 'shop_demands': {'Nochchiyagama': 100,
  'Medawachchiya': 120,
  'Medirigiriya': 85,
  'Dambulla': 150,
  'Chilaw': 50,
  'Walipennagahamula': 400},
 'vehicle_capacities': [600, 400],
 'vehicle_restricted_locations': {},
 'vehicle_restricted_roads': {},
 'same_route_locations': []}

In [65]:
data['vehicle_capacities']= {'A':600, 'B': 500}

In [66]:
data

{'distance_matrix': [[0.0, 53.53, 117.15, 76.13, 102.99, 132.7],
  [53.53, 0.0, 109.71, 79.49, 157.07, 169.49],
  [117.15, 109.71, 0.0, 57.75, 187.27, 169.94],
  [76.13, 79.49, 57.75, 0.0, 129.61, 112.28],
  [102.99, 157.07, 187.27, 129.61, 0.0, 29.79],
  [132.7, 169.49, 169.94, 112.28, 29.79, 0.0]],
 'locations': ['Nochchiyagama',
  'Medawachchiya',
  'Medirigiriya',
  'Dambulla',
  'Chilaw',
  'Walipennagahamula'],
 'depot': 0,
 'shop_demands': {'Nochchiyagama': 100,
  'Medawachchiya': 120,
  'Medirigiriya': 85,
  'Dambulla': 150,
  'Chilaw': 50,
  'Walipennagahamula': 400},
 'vehicle_capacities': {'A': 600, 'B': 500},
 'vehicle_restricted_locations': {},
 'vehicle_restricted_roads': {},
 'same_route_locations': []}

In [67]:
len(data.get('vehicle_capacities'))

2

In [68]:
list(data.get('vehicle_capacities').values())

[600, 500]

In [71]:
get_vrp_solution(data)

{'routes': {'vehicle_0': {'vehicle_id': 0,
   'route': [{'location': 'Nochchiyagama', 'current_load': 100},
    {'location': 'Chilaw', 'current_load': 150},
    {'location': 'Walipennagahamula', 'current_load': 550},
    {'location': 'Nochchiyagama', 'current_load': 550}],
   'total_distance': 263,
   'total_load': 550},
  'vehicle_1': {'vehicle_id': 1,
   'route': [{'location': 'Nochchiyagama', 'current_load': 100},
    {'location': 'Dambulla', 'current_load': 250},
    {'location': 'Medirigiriya', 'current_load': 335},
    {'location': 'Medawachchiya', 'current_load': 455},
    {'location': 'Nochchiyagama', 'current_load': 455}],
   'total_distance': 295,
   'total_load': 455}},
 'total_distance': 558,
 'total_load': 1005}